In [1]:
print("ok")


ok


In [2]:
%pwd

'c:\\Users\\aryes\\OneDrive\\Desktop\\Museum chatbot new\\Museum-assistance-chatbot\\research'

In [3]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
def load_pdf_file(data):
    loader= DirectoryLoader(data, glob= "*.pdf", loader_cls=PyPDFLoader)
    documents= loader.load()

    return documents

In [5]:
extracted_data=load_pdf_file(data=r'C:\Users\aryes\OneDrive\Desktop\Museum chatbot new\Museum-assistance-chatbot\Data')

In [6]:
# extracted_data

In [7]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [8]:
text_chunks=text_split(extracted_data)
print('Length of Text Chunks', len(text_chunks))

Length of Text Chunks 164


In [9]:
# text_chunks

In [10]:
from langchain.embeddings import HuggingFaceEmbeddings

In [11]:
def download_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [12]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
print("Model loaded successfully")

c:\Users\aryes\anaconda3\envs\museumbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Model loaded successfully


In [13]:
embeddings=download_embeddings()

C:\Users\aryes\AppData\Local\Temp\ipykernel_20304\3393384178.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [14]:
query_result=embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [15]:
# query_result

In [16]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [17]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
HUGGINGFACE_API_KEY=os.environ.get('HUGGINGFACE_API_KEY')

In [18]:
# from pinecone.grpc import PineconeGRPC as Pinecone
# from pinecone import ServerlessSpec

# pc = Pinecone(api_key=PINECONE_API_KEY)

# index_name= "museumbot"

# pc.create_index(
#     name=index_name,
#     dimension=384,
#     metric="cosine",
#     spec=ServerlessSpec(
#         cloud="aws",
#         region="us-east-1"
#     )
# )

In [19]:
import os
os.environ["PINECONE_API_KEY"]=PINECONE_API_KEY
os.environ["HUGGINGFACE_API_KEY"]=HUGGINGFACE_API_KEY

In [20]:
from langchain_pinecone import PineconeVectorStore

docsearch= PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name="museumbot",
    embedding=embeddings,
)

In [21]:
from langchain_pinecone import PineconeVectorStore

docsearch= PineconeVectorStore.from_existing_index(
    index_name="museumbot",
    embedding=embeddings
)

docsearch

In [22]:
retreiver=docsearch.as_retriever(search_type='similarity', search_kwargs={"k":3})

In [23]:
retreiver_docs= retreiver.invoke("Where is the Indian Museum Located?")
retreiver_docs

[Document(id='26ce7eeb-f9ca-453a-9858-962c1293ecab', metadata={'author': 'Sohini Pyne', 'creationdate': '2023-01-30T14:05:14+05:30', 'creator': 'Microsoft® PowerPoint® for Microsoft 365', 'moddate': '2023-01-30T14:05:14+05:30', 'page': 18.0, 'page_label': '19', 'producer': 'Microsoft® PowerPoint® for Microsoft 365', 'source': 'C:\\Users\\aryes\\OneDrive\\Desktop\\Museum chatbot new\\Museum-assistance-chatbot\\Data\\Indian museum data.pdf', 'title': 'PowerPoint Presentation', 'total_pages': 90.0}, page_content='INDIAN MUSEUM KOLKATA 19'),
 Document(id='b2de6c26-d6fe-4376-8569-3903f7b39207', metadata={'author': 'Sohini Pyne', 'creationdate': '2023-01-30T14:05:14+05:30', 'creator': 'Microsoft® PowerPoint® for Microsoft 365', 'moddate': '2023-01-30T14:05:14+05:30', 'page': 18.0, 'page_label': '19', 'producer': 'Microsoft® PowerPoint® for Microsoft 365', 'source': 'C:\\Users\\aryes\\OneDrive\\Desktop\\Museum chatbot new\\Museum-assistance-chatbot\\Data\\Indian museum data.pdf', 'title': 'Po

In [24]:
import requests

# HUGGINGFACE_API_KEY = "your_actual_huggingface_api_key"
API_URL = "https://huggingface.co/api/whoami-v2"
headers = {"Authorization": f"Bearer {HUGGINGFACE_API_KEY}"}

response = requests.get(API_URL, headers=headers)
print(response.json())  # Should return your user info if the key is correct


{'error': 'Invalid credentials in Authorization header'}


In [31]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt=(
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrived context to answer "
    "the question. if you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise. "
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [32]:
from langchain_community.llms import Ollama
from langchain.chains import RetrievalQA

llm = Ollama(model="mistral")

qa_chain=create_stuff_documents_chain(llm, prompt)
rag_chain=create_retrieval_chain(retreiver, qa_chain)


In [33]:
response= rag_chain.invoke({"input": "When was the Indian Museum Established? "})
print(response["answer"])

 The Indian Museum was established in 1814, after the Asiatic Society of Bengal was offered suitable accommodation by the Government of India in the Chowringhee-Park Street area.


In [ ]:
def chatbot():
    print("Museum Chatbot (Ollama + Mistral 7B) - Type 'exit' to quit.")
    while True:
        query = input("You: ")
        if query.lower() == "exit":
            break

        response = qa_chain.run(query)
        print("Bot:", response)

chatbot()

In [25]:
# from langchain.llms import HuggingFaceHub


# llm = HuggingFaceHub(
#     repo_id="mistralai/Mistral-7B-v0.1",  # Try this model instead
#     model_kwargs={"temperature": 0.4, "max_length": 500},
#     huggingfacehub_api_token=HUGGINGFACE_API_KEY
# )

# response = llm("When was the Indian Museum established?")
# print(response)
